# Projeto 1 - Ciência dos Dados

Nome: Filippo Ferraro

Nome: Nicolas Byung Kwan Cho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'iPad Pro.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPad Pro.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,novo ipad air destruidor demais. não precisa g...,1
1,"@beniiciojr @elizabethrsic ih, facil é só eu p...",0
2,o negócio é comprar o ipad air preto e dizer q...,1
3,"vídeo feito com realidade aumentada, alguma su...",1
4,"@drudu mas migo, n é meio parecido com o últim...",1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,rt @guiqis: novo ipad air: layout do ipad pro ...,1
1,novas cores do novo airpad air.\n\n- novo proc...,1
2,eu só quero um ipad pro,1
3,usb-c estará presente no novo ipad air.\n\num ...,1
4,"o design é baseado no ipad pro de 2018, soment...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O iPad Pro é um dispositivo do tipo tablet desenvolvido pela Apple. Por ser um produto de uma marca renomada e ter alto desempeno, o iPad Pro apresenta um valor elevado e não é um dispositivo acessível a todos.
Portanto, tweets que mencionavam o preço do produto, design, utilidade e especificações técnicas, foram considerados relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [19]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@\n@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

In [20]:
#limpando os tweets
train.Treinamento.astype(str)
train.Treinamento = train.Treinamento.apply(cleanup)
train

,Treinamento,Relevância
0,novo ipad air destruidor demais não precisa ga...,1
1,beniiciojr elizabethrsic ih facil é só eu parc...,0
2,o negócio é comprar o ipad air preto e dizer q...,1
3,vídeo feito com realidade aumentada alguma sug...,1
4,drudu mas migo n é meio parecido com o último ...,1
...,...,...
495,eu tinha feito umaudio todo lindopro 3m da joa...,0
496,alguem me da 8 mil pra eu poder comprar um ipa...,1
497,rt filipeesposito o reajuste de preços na appl...,1
498,iphone 11 não quero o 12 consigo mais votos de...,0


In [21]:
#colocando as palavras dos tweets relevantes em uma serie pd
tweets_relevantes = train.loc[train.Relevância == 1, : ]
todos_tweets_relevantes = ''
for e in tweets_relevantes.Treinamento:
    todos_tweets_relevantes += e

lista_tweets_relevantes = todos_tweets_relevantes.split()
series_tweets_relevantes = pd.Series(lista_tweets_relevantes)
series_tweets_relevantes

0             novo
1             ipad
2              air
3       destruidor
4           demais
           ...    
5987        design
5988            do
5989          ipad
5990           pro
5991           wtf
Length: 5992, dtype: object

In [22]:
#suavizacao de laplace
palavras_desconhecidas = 1000000
bonus = 2

In [9]:
#frequencias relativas das palavras em tweets relevantes (com suavizacao de laplace)
freq_tweets_relevantes = series_tweets_relevantes.value_counts() 
total_relevantes = sum(freq_tweets_relevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_relevantes = (freq_tweets_relevantes+bonus) / total_relevantes
freq_rel_tweets_relevantes

ipad                      0.000185
pro                       0.000134
o                         0.000132
e                         0.000079
que                       0.000069
                            ...   
hj                        0.000001
https//tco/6b0nc0mnywo    0.000001
chorartvshowzsx           0.000001
acabou                    0.000001
altos                     0.000001
Length: 1480, dtype: float64

In [10]:
#colocando as palavras dos tweets irrelevantes em uma serie pd
tweets_irrelevantes = train.loc[train.Relevância == 0, : ]
todos_tweets_irrelevantes = ''
for e in tweets_irrelevantes.Treinamento:
    todos_tweets_irrelevantes += e

lista_tweets_irrelevantes = todos_tweets_irrelevantes.split()
series_tweets_irrelevantes = pd.Series(lista_tweets_irrelevantes)
series_tweets_irrelevantes

0                  beniiciojr
1               elizabethrsic
2                          ih
3                       facil
4                           é
                ...          
4750                      pro
4751                      com
4752                    apple
4753                   pencil
4754    https//tco/1fvxslvwgh
Length: 4755, dtype: object

In [11]:
#frequencias relativas das palavras em tweets irrelevantes (com suavizacao de laplace)
freq_tweets_irrelevantes = series_tweets_irrelevantes.value_counts() 
total_irrelevantes = sum(freq_tweets_irrelevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_irrelevantes = (freq_tweets_irrelevantes+bonus) / total_irrelevantes
freq_rel_tweets_irrelevantes

ipad               0.000112
pro                0.000096
e                  0.000076
o                  0.000063
de                 0.000059
                     ...   
irmão              0.000001
proé               0.000001
42°                0.000001
desenharvc         0.000001
nãolove_tripper    0.000001
Length: 1418, dtype: float64

In [12]:
################################## célula desnecessária
#definindo um conjunto universo
universo = todos_tweets_relevantes + todos_tweets_irrelevantes
serie_universo = pd.Series(universo.split())
freq_rel_universo = serie_universo.value_counts(True)
freq_rel_universo
##################################

ipad               0.055183
pro                0.042527
o                  0.036013
e                  0.028569
de                 0.023078
                     ...   
pontoacabo         0.000093
saiba              0.000093
geraçãoa           0.000093
pleaseeeeeeeeem    0.000093
iphoneeee          0.000093
Length: 2354, dtype: float64

In [23]:
def classificador(tweet):
    lista_palavras = tweet.split()
    
    P_tweet_dado_relevante = 1
    P_tweet_dado_irrelevante = 1
    
    for palavra in lista_palavras:
        if palavra in freq_rel_tweets_relevantes:
            P_tweet_dado_relevante *= freq_rel_tweets_relevantes[palavra]
        else:
            P_tweet_dado_relevante *= bonus/total_relevantes
            
        if palavra in freq_rel_tweets_irrelevantes:
            P_tweet_dado_irrelevante *= freq_rel_tweets_irrelevantes[palavra]
        else:
            P_tweet_dado_irrelevante *= bonus/total_irrelevantes
            
    P_relevante = len(tweets_relevantes)/len(train)
    P_irrelevante = len(tweets_irrelevantes)/len(train)
    
    P_relevante_dado_tweet = P_tweet_dado_relevante * P_relevante
    P_irrelevante_dado_tweet = P_tweet_dado_irrelevante * P_irrelevante
    
    
    
    if P_relevante_dado_tweet > P_irrelevante_dado_tweet:
        diferenca = (P_relevante_dado_tweet-P_irrelevante_dado_tweet)
        if diferenca > 10e-80:
            return 5
        elif diferenca <10e-200:
            return 3
        else:
            return 4
    else:
        diferenca = (P_irrelevante_dado_tweet-P_relevante_dado_tweet)
        if diferenca > 10e-80:
            return 1
        elif diferenca <10e-200:
            return 3
        else:
            return 2

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [24]:
#limpando os tweets
test.Teste.astype(str)
test.Teste = test.Teste.apply(cleanup)
test

,Teste,Relevância,Classificador
0,rt guiqis novo ipad air layout do ipad pro com...,1,muito relevante
1,novas cores do novo airpad air novo processado...,1,relevante
2,eu só quero um ipad pro,1,muito relevante
3,usbc estará presente no novo ipad airum ipad m...,1,relevante
4,o design é baseado no ipad pro de 2018 somente...,1,neutro
...,...,...,...
129,minha bamboo que tenho desde 2011 tá querendo ...,1,neutro
130,a capa pro meu ipad chegou aleluia não aguenta...,0,muito irrelevante
131,blogdoiphone compensa comprar o ipad pro ou va...,1,relevante
132,xandcasusao eu fui eu tava a minha era um pand...,0,relevante


In [25]:
lista_classificador = []
for tweet in test.Teste:
    lista_classificador.append(classificador(tweet))
    
series_classificador = pd.Series(lista_classificador)
series_classificador

0      5
1      4
2      5
3      4
4      3
      ..
129    3
130    1
131    4
132    4
133    5
Length: 134, dtype: int64

In [26]:
test['Classificador'] = series_classificador
test['Classificador'] = test['Classificador'].astype('category')
test['Classificador'].cat.categories = ['muito irrelevante','irrelevante','neutro','relevante','muito relevante']
test

,Teste,Relevância,Classificador
0,rt guiqis novo ipad air layout do ipad pro com...,1,muito relevante
1,novas cores do novo airpad air novo processado...,1,relevante
2,eu só quero um ipad pro,1,muito relevante
3,usbc estará presente no novo ipad airum ipad m...,1,relevante
4,o design é baseado no ipad pro de 2018 somente...,1,neutro
...,...,...,...
129,minha bamboo que tenho desde 2011 tá querendo ...,1,neutro
130,a capa pro meu ipad chegou aleluia não aguenta...,0,muito irrelevante
131,blogdoiphone compensa comprar o ipad pro ou va...,1,relevante
132,xandcasusao eu fui eu tava a minha era um pand...,0,relevante


In [28]:
comparacao = pd.crosstab(test.Relevância, test.Classificador)
comparacao

Classificador,muito irrelevante,irrelevante,neutro,relevante,muito relevante
Relevância,,,,,
0,3,10,10,22,12
1,1,0,9,39,28


___
### Concluindo

Boa parte dos tweets com relevância 0 surpreendentemente foi classificada como relevante ou muito relevante, já os tweets com relevância 1 seguiram de acordo com o esperado e apenas uma pequena porcentagem foi classificada como muito irrelevante.

Mensagens que apresentam sarcasmo são entendidas pelo classificador como o sentido literal das palavras e não seu oposto. Assim, tweets que desejam expressar algo irrelevante mas utiliza palavras relevantes são classificados como relevantes.

Este trabalho deve continuar sendo financiado pois, por enquanto, o classificador está funcionando apenas com uma base de treinamento e já apresenta bons resultados. Visto isso, planejamos aperfeiçoar e preparar o projeto para que seja capaz de alimentar sua base automaticamente usando o próprio classificador, aplicado a novos tweets.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [18]:
1e-5+5

5.00001